# Trust but Verify - Inspection of Large Image Collections

This notebook and accompanying [Python script characterize_data.py](characterize_data.py) illustrate the use of SimpleITK as a tool for efficient data inspection on large image collections, as part of familiarizing oneself with the data and performing cleanup prior to its use in deep learning or any other supervised machine learning approach.

The reasons for inspecting your data before using it include:
1. Identification of corrupt images.
2. Identification of erroneous images (label noise).
3. Assessment of data quality and variability in terms of intensity range, image resolution, and pixel types.
4. Reduction of workload, identifying redundant information content (e.g. a greyscale/single channel image masquerading as a color/three channel image - think x-ray in jpg file).


In [1]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
import ipympl
from DicomHelper import *

# utility method that either downloads data from the Girder repository or
# if already downloaded returns the file name for reading from disk (cached data)
OUTPUT_DIR = "OUTPUT-ALL-UNLABELED"



In [2]:
%env SITK_SHOW_COMMAND /Applications/Slicer.app/Contents/MacOS/Slicer

env: SITK_SHOW_COMMAND=/Applications/Slicer.app/Contents/MacOS/Slicer


In [3]:
%matplotlib widget

In [4]:
data_root_dir = "/Users/seanreed/Documents/ALL-UNLABELED-DATA"

In [5]:
!ls $data_root_dir

1.2.276.0.2783747.3.1.2.1150183542.91496.1584557269.16654
1.2.276.0.2783747.3.1.2.1150183542.91496.1584560923.16663
1.2.276.0.7230010.3.1.2.1498839191.5144.1567777520.295
1.2.276.0.7230010.3.1.2.2733322860.4416.1572435955.336
1.2.276.0.7230010.3.1.2.738517100.4712.1625692712.6881
1.2.276.0.7230010.3.1.2.738517100.4712.1625694761.7821
1.2.276.0.7230010.3.1.2.738517100.4712.1625695650.8766
1.2.276.0.7230010.3.1.2.738517100.5904.1625697387.543
1.2.276.0.7230010.3.1.2.738517100.7560.1611692908.810
1.2.276.0.7230010.3.1.2.738517100.7560.1611699852.1756
1.2.392.200036.9116.2.2.2.1762671078.1544058796.778235
1.2.392.200036.9116.2.2.2.1762671078.1544062790.854133
1.2.392.200036.9116.2.2.2.1762671078.1544064007.853456
1.2.392.200036.9116.2.2.2.1762671078.1544065347.70543
1.2.392.200036.9116.2.2.2.1762671078.1544071914.125853
1.2.392.200036.9116.2.2.2.1762671078.1544074601.48209
1.2.392.200036.9116.2.2.2.1762671078.1544578457.236922
1.2.392.200036.9116.2.2.2.1762671078.1555384900.905993
1.2.392.

## Characterizing  image set

To characterize the image set we have written a [Python script](characterize_data.py) that you should run from the command line. This script is very flexible and allows you to robustly characterize your image set. Try the various options and learn more about your data. You'd be surprised how many times the data isn't what you thought it is when only relying on visual inspection. The script allows you to inspect your data both on a file by file basis and as DICOM series where an image (volume) is stored in multiple files.

DICOM series:
```
python characterize_data.py data output/DICOM_image_data_report.csv per_series \
--metadata_keys "0008|0060" "0018|5101" --metadata_keys_headings "modality" "radiographic view"  
```


After characterizing the image set we turn to visual inspection. 

In [ ]:
!python characterize_data.py --dir "/Users/seanreed/Documents/ALL-UNLABELED-DATA/1.2.276.0.7230010.3.1.2.1498839191.5144.1567777520.295" --output "./OUTPUT-ALL-UNLABELED/output_one_study.csv" --metadata_keys "0008|0060" "0018|5101" --metadata_keys_headings "modality" "radiographic view" 


In [6]:
!python characterize_data.py --dir $data_root_dir --output "./OUTPUT-ALL-UNLABELED/output_all_unlabeled.csv" --metadata_keys "0008|0060" "0018|5101" --metadata_keys_headings "modality" "radiographic view" 

args are 
 Namespace(analyze='per_series', dir='/Users/seanreed/Documents/ALL-UNLABELED-DATA', external_applications=[], external_applications_headings=[], imageIO='', metadata_keys=['0008|0060', '0018|5101'], metadata_keys_headings=['modality', 'radiographic view'], output='./OUTPUT-ALL-UNLABELED/output_all_unlabeled.csv')
ImageSeriesReader (0x7fc597184dc0): Non uniform sampling or missing slices detected,  maximum nonuniformity:44.7

ImageSeriesReader (0x7fc597172d30): Non uniform sampling or missing slices detected,  maximum nonuniformity:28.4194

ImageSeriesReader (0x7fc5967ede40): Non uniform sampling or missing slices detected,  maximum nonuniformity:47.5

ImageSeriesReader (0x7fc5971a5ee0): Non uniform sampling or missing slices detected,  maximum nonuniformity:16.9483

ImageSeriesReader (0x7fc597b56df0): Non uniform sampling or missing slices detected,  maximum nonuniformity:19.3538

ImageSeriesReader (0x7fc5971a5ff0): Non uniform sampling or missing slices detected,  maximum n

ImageSeriesReader (0x7fc5967ede40): Non uniform sampling or missing slices detected,  maximum nonuniformity:10.4528

ImageSeriesReader (0x7fc5971a4cf0): Non uniform sampling or missing slices detected,  maximum nonuniformity:3.22388

ImageSeriesReader (0x7fc597b20980): Non uniform sampling or missing slices detected,  maximum nonuniformity:8.2

ImageSeriesReader (0x7fc5971a56c0): Non uniform sampling or missing slices detected,  maximum nonuniformity:13.4328

ImageSeriesReader (0x7fc597b617c0): Non uniform sampling or missing slices detected,  maximum nonuniformity:18

ImageSeriesReader (0x7fc5967ede40): Non uniform sampling or missing slices detected,  maximum nonuniformity:12.9167

ImageSeriesReader (0x7fc597172d30): Non uniform sampling or missing slices detected,  maximum nonuniformity:9.63636

ImageSeriesReader (0x7fc5967ede40): Non uniform sampling or missing slices detected,  maximum nonuniformity:12.8571

ImageSeriesReader (0x7fc59aa05e60): Non uniform sampling or missing slice

ImageSeriesReader (0x7fc597172d30): Non uniform sampling or missing slices detected,  maximum nonuniformity:41.7692

ImageSeriesReader (0x7fc5967ede40): Non uniform sampling or missing slices detected,  maximum nonuniformity:52.3333

ImageSeriesReader (0x7fc59aa050e0): Non uniform sampling or missing slices detected,  maximum nonuniformity:1091.3

ImageSeriesReader (0x7fc59aa050e0): Non uniform sampling or missing slices detected,  maximum nonuniformity:1106.18

ImageSeriesReader (0x7fc59aa04460): Non uniform sampling or missing slices detected,  maximum nonuniformity:129.075

ImageSeriesReader (0x7fc597172d30): Non uniform sampling or missing slices detected,  maximum nonuniformity:11.8491

ImageSeriesReader (0x7fc597172d30): Non uniform sampling or missing slices detected,  maximum nonuniformity:11.8318

ImageSeriesReader (0x7fc5967ede40): Non uniform sampling or missing slices detected,  maximum nonuniformity:28.8689

ImageSeriesReader (0x7fc593044930): Non uniform sampling or missi

ImageSeriesReader (0x7fc597172d30): Non uniform sampling or missing slices detected,  maximum nonuniformity:10.2778

ImageSeriesReader (0x7fc597b3a580): Non uniform sampling or missing slices detected,  maximum nonuniformity:8.33333

ImageSeriesReader (0x7fc5971583c0): Non uniform sampling or missing slices detected,  maximum nonuniformity:4

ImageSeriesReader (0x7fc591ea59d0): Non uniform sampling or missing slices detected,  maximum nonuniformity:59.8889

ImageSeriesReader (0x7fc59aa04730): Non uniform sampling or missing slices detected,  maximum nonuniformity:20.4348

ImageSeriesReader (0x7fc59aa04080): Non uniform sampling or missing slices detected,  maximum nonuniformity:18.8667

ImageSeriesReader (0x7fc597b50ec0): Non uniform sampling or missing slices detected,  maximum nonuniformity:69

ImageSeriesReader (0x7fc593047d30): Non uniform sampling or missing slices detected,  maximum nonuniformity:21.25

ImageSeriesReader (0x7fc5967ede40): Non uniform sampling or missing slices de

ImageSeriesReader (0x7fc5967ede40): Non uniform sampling or missing slices detected,  maximum nonuniformity:88.0909

ImageSeriesReader (0x7fc597b5bb00): Non uniform sampling or missing slices detected,  maximum nonuniformity:13.1613

ImageSeriesReader (0x7fc591ea5180): Non uniform sampling or missing slices detected,  maximum nonuniformity:6.25

ImageSeriesReader (0x7fc5967ede40): Non uniform sampling or missing slices detected,  maximum nonuniformity:4.80851

ImageSeriesReader (0x7fc5967ede40): Non uniform sampling or missing slices detected,  maximum nonuniformity:13.6712

ImageSeriesReader (0x7fc597172d30): Non uniform sampling or missing slices detected,  maximum nonuniformity:11.5238

ImageSeriesReader (0x7fc597b5bb00): Non uniform sampling or missing slices detected,  maximum nonuniformity:10.9841

ImageSeriesReader (0x7fc593044ca0): Non uniform sampling or missing slices detected,  maximum nonuniformity:12.36

ImageSeriesReader (0x7fc597b20980): Non uniform sampling or missing s

...

ImageSeriesReader (0x7fc5967ede40): Non uniform sampling or missing slices detected,  maximum nonuniformity:18.5246

ImageSeriesReader (0x7fc597b15e90): Non uniform sampling or missing slices detected,  maximum nonuniformity:48.6

ImageSeriesReader (0x7fc597b59510): Non uniform sampling or missing slices detected,  maximum nonuniformity:43

ImageSeriesReader (0x7fc597b3a580): Non uniform sampling or missing slices detected,  maximum nonuniformity:50.5

ImageSeriesReader (0x7fc5967ede40): Non uniform sampling or missing slices detected,  maximum nonuniformity:2.09543

ImageSeriesReader (0x7fc591e81080): Non uniform sampling or missing slices detected,  maximum nonuniformity:8.56311

ImageSeriesReader (0x7fc591e81080): Non uniform sampling or missing slices detected,  maximum nonuniformity:355.837

ImageSeriesReader (0x7fc591ea59d0): Non uniform sampling or missing slices detected,  maximum nonuniformity:352.376

ImageSeriesReader (0x7fc597b50ec0): Non uniform sampling or missing slices 

ImageSeriesReader (0x7fc597b15e90): Non uniform sampling or missing slices detected,  maximum nonuniformity:11.4286

ImageSeriesReader (0x7fc59304a360): Non uniform sampling or missing slices detected,  maximum nonuniformity:33.0526

ImageSeriesReader (0x7fc597b15e90): Non uniform sampling or missing slices detected,  maximum nonuniformity:59.3333

ImageSeriesReader (0x7fc392304900): Non uniform sampling or missing slices detected,  maximum nonuniformity:74.5714

ImageSeriesReader (0x7fc392306b10): Non uniform sampling or missing slices detected,  maximum nonuniformity:7.68831

ImageSeriesReader (0x7fc597b15e90): Non uniform sampling or missing slices detected,  maximum nonuniformity:17.1304

ImageSeriesReader (0x7fc597b59510): Non uniform sampling or missing slices detected,  maximum nonuniformity:22.7664

ImageSeriesReader (0x7fc591ea9380): Non uniform sampling or missing slices detected,  maximum nonuniformity:17.1795

ImageSeriesReader (0x7fc392308b10): Non uniform sampling or miss

ImageSeriesReader (0x7fc3923040b0): Non uniform sampling or missing slices detected,  maximum nonuniformity:17.5227

ImageSeriesReader (0x7fc3923081b0): Non uniform sampling or missing slices detected,  maximum nonuniformity:21.1892

ImageSeriesReader (0x7fc392304080): Non uniform sampling or missing slices detected,  maximum nonuniformity:25.8679

ImageSeriesReader (0x7fc597b15e90): Non uniform sampling or missing slices detected,  maximum nonuniformity:14.3684

ImageSeriesReader (0x7fc392305cd0): Non uniform sampling or missing slices detected,  maximum nonuniformity:20.2812

ImageSeriesReader (0x7fc591ea8910): Non uniform sampling or missing slices detected,  maximum nonuniformity:8.375

ImageSeriesReader (0x7fc597b15e90): Non uniform sampling or missing slices detected,  maximum nonuniformity:11.9091

ImageSeriesReader (0x7fc597b15e90): Non uniform sampling or missing slices detected,  maximum nonuniformity:7.90909

ImageSeriesReader (0x7fc591ea9270): Non uniform sampling or missin

ImageSeriesReader (0x7fc392306230): Non uniform sampling or missing slices detected,  maximum nonuniformity:8.58182

ImageSeriesReader (0x7fc392306340): Non uniform sampling or missing slices detected,  maximum nonuniformity:10.2642

ImageSeriesReader (0x7fc5967d3820): Non uniform sampling or missing slices detected,  maximum nonuniformity:6.40741

ImageSeriesReader (0x7fc392305030): Non uniform sampling or missing slices detected,  maximum nonuniformity:19.6

ImageSeriesReader (0x7fc597b50ec0): Non uniform sampling or missing slices detected,  maximum nonuniformity:12.1935

ImageSeriesReader (0x7fc392304fc0): Non uniform sampling or missing slices detected,  maximum nonuniformity:13.8167

ImageSeriesReader (0x7fc597b64570): Non uniform sampling or missing slices detected,  maximum nonuniformity:8.66

ImageSeriesReader (0x7fc597b15e90): Non uniform sampling or missing slices detected,  maximum nonuniformity:10.9783

ImageSeriesReader (0x7fc597b15e90): Non uniform sampling or missing sl

ImageSeriesReader (0x7fc597b15e90): Non uniform sampling or missing slices detected,  maximum nonuniformity:10.2462

ImageSeriesReader (0x7fc597b59510): Non uniform sampling or missing slices detected,  maximum nonuniformity:105.73

ImageSeriesReader (0x7fc392308190): Non uniform sampling or missing slices detected,  maximum nonuniformity:54

ImageSeriesReader (0x7fc392307060): Non uniform sampling or missing slices detected,  maximum nonuniformity:103.304

ImageSeriesReader (0x7fc597b1e0b0): Non uniform sampling or missing slices detected,  maximum nonuniformity:138.474

ImageSeriesReader (0x7fc597b5bb00): Non uniform sampling or missing slices detected,  maximum nonuniformity:15.1897

ImageSeriesReader (0x7fc597b15e90): Non uniform sampling or missing slices detected,  maximum nonuniformity:11.825

Traceback (most recent call last):
  File "characterize_data.py", line 586, in <module>
    sys.exit(main())
  File "characterize_data.py", line 492, in main
    df.to_csv(args.output, ind

## Now we look at our data using the DICOM series based approach.

After selecting our images of interest we print the associated files. Notice that for the series based approach for some images there is a single file association and for some multiple files.

In [7]:
### unlabeled data
data_root_dir = "/Users/seanreed/Documents/ALL-UNLABELED-DATA"

In [ ]:
faux_series_volume_file_name = os.path.join(OUTPUT_DIR, "faux_series_volume.pkl")
faux_series_file_list_name = os.path.join(OUTPUT_DIR, "faux_series_file_list.pkl")
faux_volume_image_files, image_file_list = visualize_series(
    data_root_dir, projection_axis=2, thumbnail_size=[64, 64], tile_size=[5, 5]
)
with open(faux_series_volume_file_name, "wb") as fp:
    pickle.dump(faux_volume_image_files, fp)
with open(faux_series_file_list_name, "wb") as fp:
    pickle.dump(image_file_list, fp)

GDCMSeriesFileNames (0x7fab515aa740): No Series were found

GDCMSeriesFileNames (0x7fab5123fe70): No Series were found

GDCMSeriesFileNames (0x7fab5123fe70): No Series were found

GDCMSeriesFileNames (0x7fab515aa740): No Series were found

GDCMSeriesFileNames (0x7fab4aa5b260): No Series were found

GDCMSeriesFileNames (0x7fab5123fe70): No Series were found

GDCMSeriesFileNames (0x7fab515aa740): No Series were found

GDCMSeriesFileNames (0x7fab5123fe70): No Series were found

GDCMSeriesFileNames (0x7fab515aa740): No Series were found

GDCMSeriesFileNames (0x7fab5123fe70): No Series were found

GDCMSeriesFileNames (0x7fab5123fe70): No Series were found

GDCMSeriesFileNames (0x7fab5123fe70): No Series were found

GDCMSeriesFileNames (0x7fab5123fe70): No Series were found

GDCMSeriesFileNames (0x7fab5123fe70): No Series were found

GDCMSeriesFileNames (0x7fab515aa740): No Series were found

GDCMSeriesFileNames (0x7fab5123fe70): No Series were found

ImageSeriesReader (0x7fab51215410): Non 

ImageSeriesReader (0x7fab515af660): Non uniform sampling or missing slices detected,  maximum nonuniformity:16.1053

ImageSeriesReader (0x7fab52d35360): Non uniform sampling or missing slices detected,  maximum nonuniformity:18.9041

ImageSeriesReader (0x7fab4e3b85e0): Non uniform sampling or missing slices detected,  maximum nonuniformity:22.9608

ImageSeriesReader (0x7faaa750ec10): Non uniform sampling or missing slices detected,  maximum nonuniformity:22.2302

ImageSeriesReader (0x7fab515a8800): Non uniform sampling or missing slices detected,  maximum nonuniformity:16.0299

ImageSeriesReader (0x7fab51264b60): Non uniform sampling or missing slices detected,  maximum nonuniformity:12.8906

ImageSeriesReader (0x7fab49c943a0): Non uniform sampling or missing slices detected,  maximum nonuniformity:12.9375

ImageSeriesReader (0x7faaa750e370): Non uniform sampling or missing slices detected,  maximum nonuniformity:6.71014

ImageSeriesReader (0x7fab52d17ab0): Non uniform sampling or miss

ImageSeriesReader (0x7fab4e32c090): Non uniform sampling or missing slices detected,  maximum nonuniformity:39

ImageSeriesReader (0x7fab52d0cb60): Non uniform sampling or missing slices detected,  maximum nonuniformity:32.8333

ImageSeriesReader (0x7fab4e32c090): Non uniform sampling or missing slices detected,  maximum nonuniformity:44.6

ImageSeriesReader (0x7fab51225480): Non uniform sampling or missing slices detected,  maximum nonuniformity:61.7143

ImageSeriesReader (0x7fab52d18a60): Non uniform sampling or missing slices detected,  maximum nonuniformity:19.2727

ImageSeriesReader (0x7fab52d4bda0): Non uniform sampling or missing slices detected,  maximum nonuniformity:23

ImageSeriesReader (0x7fab52d36100): Non uniform sampling or missing slices detected,  maximum nonuniformity:40

ImageSeriesReader (0x7fab49c9c8a0): Non uniform sampling or missing slices detected,  maximum nonuniformity:86.625

ImageSeriesReader (0x7fab515af660): Non uniform sampling or missing slices detected

ImageSeriesReader (0x7fab4e32c090): Non uniform sampling or missing slices detected,  maximum nonuniformity:10.0556

ImageSeriesReader (0x7fab515af660): Non uniform sampling or missing slices detected,  maximum nonuniformity:16.1622

ImageSeriesReader (0x7fab52d63cf0): Non uniform sampling or missing slices detected,  maximum nonuniformity:24.1562

ImageSeriesReader (0x7fab515b2030): Non uniform sampling or missing slices detected,  maximum nonuniformity:12.3019

ImageSeriesReader (0x7fab52d0f700): Non uniform sampling or missing slices detected,  maximum nonuniformity:13.3205

ImageSeriesReader (0x7fab51242210): Non uniform sampling or missing slices detected,  maximum nonuniformity:34.2222

ImageSeriesReader (0x7fab4ff12210): Non uniform sampling or missing slices detected,  maximum nonuniformity:31.4545

ImageSeriesReader (0x7fab52d63cf0): Non uniform sampling or missing slices detected,  maximum nonuniformity:35.8182

ImageSeriesReader (0x7faaa7511e70): Non uniform sampling or miss

ImageSeriesReader (0x7faaa7505290): Non uniform sampling or missing slices detected,  maximum nonuniformity:1.5868

ImageSeriesReader (0x7fab515b2030): Non uniform sampling or missing slices detected,  maximum nonuniformity:8.33058

ImageSeriesReader (0x7fab4e3c0a70): Non uniform sampling or missing slices detected,  maximum nonuniformity:8.14865

ImageSeriesReader (0x7faaa7510af0): Non uniform sampling or missing slices detected,  maximum nonuniformity:71.1304

ImageSeriesReader (0x7faaa7517640): Non uniform sampling or missing slices detected,  maximum nonuniformity:15.9167

ImageSeriesReader (0x7faaa7511e70): Non uniform sampling or missing slices detected,  maximum nonuniformity:11.75

ImageSeriesReader (0x7faaa75159d0): Non uniform sampling or missing slices detected,  maximum nonuniformity:19

ImageSeriesReader (0x7fab52d63cf0): Non uniform sampling or missing slices detected,  maximum nonuniformity:11

ImageSeriesReader (0x7fab52d597d0): Non uniform sampling or missing slices de

ImageSeriesReader (0x7fab49c99990): Non uniform sampling or missing slices detected,  maximum nonuniformity:3.37779

ImageSeriesReader (0x7faa7a405190): Non uniform sampling or missing slices detected,  maximum nonuniformity:1.37329

ImageSeriesReader (0x7fab515cc630): Non uniform sampling or missing slices detected,  maximum nonuniformity:1.37797

ImageSeriesReader (0x7faaed48c180): Non uniform sampling or missing slices detected,  maximum nonuniformity:60.6667

ImageSeriesReader (0x7faaed48b7a0): Non uniform sampling or missing slices detected,  maximum nonuniformity:174.286

ImageSeriesReader (0x7faaa7509a30): Non uniform sampling or missing slices detected,  maximum nonuniformity:11.7181

ImageSeriesReader (0x7faaa75146d0): Non uniform sampling or missing slices detected,  maximum nonuniformity:11.7838

ImageSeriesReader (0x7faaa75080b0): Non uniform sampling or missing slices detected,  maximum nonuniformity:12.7438

ImageSeriesReader (0x7fab49ca26a0): Non uniform sampling or miss

ImageSeriesReader (0x7fab515efd60): Non uniform sampling or missing slices detected,  maximum nonuniformity:21.2237

ImageSeriesReader (0x7faaa75107f0): Non uniform sampling or missing slices detected,  maximum nonuniformity:14.046

ImageSeriesReader (0x7faa7d604080): Non uniform sampling or missing slices detected,  maximum nonuniformity:33.5333

ImageSeriesReader (0x7fab515d9da0): Non uniform sampling or missing slices detected,  maximum nonuniformity:19.2667

ImageSeriesReader (0x7faaed4acdd0): Non uniform sampling or missing slices detected,  maximum nonuniformity:47.5556

ImageSeriesReader (0x7faaa75a2f40): Non uniform sampling or missing slices detected,  maximum nonuniformity:23.1

ImageSeriesReader (0x7fab49cd4f00): Non uniform sampling or missing slices detected,  maximum nonuniformity:10

ImageSeriesReader (0x7faaed4acdd0): Non uniform sampling or missing slices detected,  maximum nonuniformity:7.03448

ImageSeriesReader (0x7fab515d27a0): Non uniform sampling or missing slice

ImageSeriesReader (0x7fab512fa2f0): Non uniform sampling or missing slices detected,  maximum nonuniformity:15.6897

ImageSeriesReader (0x7fab49ca26a0): Non uniform sampling or missing slices detected,  maximum nonuniformity:12.4337

ImageSeriesReader (0x7fab54228d60): Non uniform sampling or missing slices detected,  maximum nonuniformity:12.506

ImageSeriesReader (0x7faaed4acdd0): Non uniform sampling or missing slices detected,  maximum nonuniformity:7.29897

ImageSeriesReader (0x7faaed4b7c80): Non uniform sampling or missing slices detected,  maximum nonuniformity:12.9

ImageSeriesReader (0x7fab515e3460): Non uniform sampling or missing slices detected,  maximum nonuniformity:5.28481

ImageSeriesReader (0x7fab515efd60): Non uniform sampling or missing slices detected,  maximum nonuniformity:38.8

ImageSeriesReader (0x7faaed4acdd0): Non uniform sampling or missing slices detected,  maximum nonuniformity:42.8

ImageSeriesReader (0x7faaed4acdd0): Non uniform sampling or missing slices

ImageSeriesReader (0x7faaa75a8640): Non uniform sampling or missing slices detected,  maximum nonuniformity:128.053

ImageSeriesReader (0x7faaa7509650): Non uniform sampling or missing slices detected,  maximum nonuniformity:6.83117

ImageSeriesReader (0x7faaed48bcc0): Non uniform sampling or missing slices detected,  maximum nonuniformity:10.5152

ImageSeriesReader (0x7fab515fd3e0): Non uniform sampling or missing slices detected,  maximum nonuniformity:6.41176

ImageSeriesReader (0x7fab515d2fc0): Non uniform sampling or missing slices detected,  maximum nonuniformity:65.2

ImageSeriesReader (0x7faaed48bcc0): Non uniform sampling or missing slices detected,  maximum nonuniformity:35.4

ImageSeriesReader (0x7fab515f0330): Non uniform sampling or missing slices detected,  maximum nonuniformity:37

ImageSeriesReader (0x7faaed47cd30): Non uniform sampling or missing slices detected,  maximum nonuniformity:12.25

ImageSeriesReader (0x7faaed47cd30): Non uniform sampling or missing slices de

In [ ]:
print(image_file_list[:2])
print(len(image_file_list))
for i, sub_list in enumerate(image_file_list):
    print(f"sub_list{i} has {len(sub_list)} dicom files")

In [ ]:
with open(faux_series_volume_file_name, "rb") as fp:
    faux_volume_image_files = pickle.load(fp)
with open(faux_series_file_list_name, "rb") as fp:
    image_file_list = pickle.load(fp)

try:
    image_selection_gui2 = ImageSelection(
        faux_volume_image_files,
        image_file_list,
        figure_size=(10, 5),
        selection_func=show_image,
    )
except Exception as e:
    print(e)

In [ ]:
selected_files = image_selection_gui2.get_selected_images()
print(selected_files)